In [1]:
import calendar
import time
import requests
import datetime
from bs4 import BeautifulSoup
from urllib import parse
import csv


In [2]:
# get html code, convert to plaintext
def getHTML(url):
    r = requests.get(url,verify=False)
    #r.status_code
    plain=r.text
    s = BeautifulSoup(plain, 'html.parser')
    return s

In [3]:
def getForum(s):
    fields = ['fid', 'f_url','f_heading','f_subheading','num_topics','num_posts','last_post_date','last_post_uid']
    f_post=s.find_all('tr',{'class':'post'})
    rows = []
    for tags in f_post:
        link = tags.find_all('span',{'class':'forumheading'})[0]
        a = link.find_all('a')[0]
        fname = a.contents[0]
        url = a['href']
        fid = parse.parse_qs(parse.urlsplit(url).query)['f'][0]
        
        subheading = tags.find_all('span',{'class':'subforumheading'})[0].contents[0]
        topics = tags.find_all('td')[2].contents[0]
        posts = tags.find_all('td')[3].contents[0]
        last_date = tags.find_all('td')[4].contents[0]
        if last_date != '-':
            uid = parse.parse_qs(parse.urlsplit(tags.find_all('td')[4].contents[3]['data-remote']).query)['uid'][0]
        else:
            uid = '-'

        row = [fid, url, fname,subheading,topics,posts,last_date,uid]
        rows.append(row)

    return fields, rows

def getTopics(s, fid):
    fields = ['fid','tid','t_url','t_heading','num_replies','num_views','starter_uid','starter_name','last_post_date']
    rows = []
    postlst = s.find_all('tr',{'class':'post'})
    for post in postlst:
        # print('post: ', post)
        a = post.find_all('a')[0]
        # print('a:',a)
        tname = a.contents[0]
        url = a['href']
        # print('url: ', url)
        tid = parse.parse_qs(parse.urlsplit(url).query)['t'][0]

        uname = post.find_all('a')[1].contents[0]
        uid = parse.parse_qs(parse.urlsplit(post.find_all('a')[1]['data-remote']).query)['uid'][0]
        
        rep = post.find_all('td')[3].contents[0]
        view = post.find_all('td')[4].contents[0]
        date = post.find_all('td')[5].contents[0]
        
        row = [fid,tid,url,tname,rep,view,uid,uname,date]
        rows.append(row)
    return fields, rows


In [6]:
def writeToCSV(field, rows, fname):
    with open(fname, 'w') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerow(field)
        csvwriter.writerows(rows)
    f.close()

In [9]:
topsoup = getHTML('https://www.alzconnected.org/discussion.aspx')
f_fields, f_rows = getForum(topsoup)
writeToCSV(f_fields, f_rows,'forum.csv')

In [10]:
#f_rows = [f_rows[2]]
for row in f_rows:
    print('getting info from forum: {}...'.format(row[2]))
    rows = []
    i = 1
    while True:
        forumURL = "https://www.alzconnected.org{}&page={}".format(row[1],i)
        s = getHTML(forumURL)
        fields, cur_row = getTopics(s,row[0])
        if len(cur_row)==0:
            print('Done with forum: {}.'.format(row[2]))
            break
        else:
            if i % 10 == 0:
                print('Successfully getting info from {} page {}'.format(row[2],i))
            rows.extend(cur_row)
            i+=1
    writeToCSV (fields, rows, 'topics_{}.csv'.format(row[2]))

getting info from forum: Caregivers Forum...
Successfully getting info from Caregivers Forum page 10
Successfully getting info from Caregivers Forum page 20
Successfully getting info from Caregivers Forum page 30
Successfully getting info from Caregivers Forum page 40
Successfully getting info from Caregivers Forum page 50
Successfully getting info from Caregivers Forum page 60
Successfully getting info from Caregivers Forum page 70
Successfully getting info from Caregivers Forum page 80
Successfully getting info from Caregivers Forum page 90
Successfully getting info from Caregivers Forum page 100
Successfully getting info from Caregivers Forum page 110
Successfully getting info from Caregivers Forum page 120
Successfully getting info from Caregivers Forum page 130
Successfully getting info from Caregivers Forum page 140
Successfully getting info from Caregivers Forum page 150
Successfully getting info from Caregivers Forum page 160
Successfully getting info from Caregivers Forum page

------------------------

In [34]:
def getPostsAndUser(s):
    post_fields = ['tid', 'pid','p_content','is_start']
    user_fields = ['uid','join_date','num_post']
    postlst = s.find_all('td',{'class':'message ekMessage'})
    userlst = s.find_all('tr',{'class':'postheader'})


In [32]:
soup.find_all('td',{'class':'message ekMessage'})[1]

<td class="message ekMessage">
<strong>I'm not quite certain of your question.  Are you asking if you should have more neurocognitive testing?  Do you have a diagnosis?</strong><div><strong><br/></strong><div><strong><br/></strong></div>
<div><strong>Repeated neuropsychological evaluations can document the progress or absence of progress.  In my case, full neurocognitive testing about a year after beginning medications documented a small decline in some areas, improvement in other areas, and no change in other areas.  The next year there was no significant decline documented.  This, along with the negative Amyvid PET scan, confirmed to my neurologist that I do not have Alzheimer's Disease.  </strong></div>
<div><div> </div>
<div><div><div><strong><br/></strong></div>
<div><strong>Iris</strong></div>
</div>
</div>
</div>
</div><br/>
</td>

In [33]:
soup.find_all('tr',{'class':'postheader'})[1]

<tr class="postheader">
<td colspan="2" width="140px"><a name="post2148086137"></a>
<b><a data-refresh="true" data-remote="viewprofile.aspx?uid=2147484334" data-toggle="modal" href="#profile-modal" target="_self">Iris L.</a></b>
</td>
<td width="80%">
<table cellpadding="0" cellspacing="0" width="100%">
<tr>
<td class="postheader">
<b>Posted:</b> 
				Monday, February 15, 2021 5:14 PM			</td>
<td align="right" class="postheader">
<div class="ekForumButtonWrapper">
</div>
</td>
</tr>
</table>
</td>
</tr>